In [2]:
from aicsimageio import AICSImage
from tifffile import imsave
from pathlib import Path
import numpy as np
from skimage.filters import threshold_otsu
from skimage.morphology import ball, binary_closing, remove_small_objects, dilation, erosion, disk
from aicssegmentation.core.pre_processing_utils import  intensity_normalization, edge_preserving_smoothing_3d
from aicssegmentation.core.vessel import vesselness3D
from aicssegmentation.core.seg_dot import dot_2d
from aicssegmentation.core.utils import topology_preserving_thinning, hole_filling
import matplotlib.pyplot as plt
import pandas as pd
from itkwidgets import view   
import seaborn as sns
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
plt.rcParams["figure.figsize"] = [16, 12]

<h3> Example 1 <h3>

In [4]:
# Comparing example 1 (50ms)
filename = "C:/Users/sandra.oluoch/Desktop/slurm-down/golgi-50ms-ex1v8.tiff"
reader = AICSImage(filename)
IMG_ex1_50ms = reader.data.astype(np.float32)
#IMG_ex1_50ms

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 0


In [7]:
# Comparing example 1 (200ms)
filename = "C:/Users/sandra.oluoch/Desktop/slurm-down/golgi-200ms-ex1v8.1.tiff"
reader = AICSImage(filename)
IMG_ex1_200ms = reader.data.astype(np.float32)
#IMG_ex1_200ms

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 0


In [9]:
ex1_similarity =2*np.sum(~np.logical_xor(IMG_ex1_50ms,IMG_ex1_200ms))/(IMG_ex1_50ms.size+IMG_ex1_200ms.size)
ex1_similarity

0.9999471296296296

<h3> Example 2 <h3>

In [10]:
# Comparing example 2 (50ms)
filename = "C:/Users/sandra.oluoch/Desktop/slurm-down/golgi-50ms-ex1v8.tiff"
reader = AICSImage(filename)
IMG_ex2_50ms = reader.data.astype(np.float32)

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 0


In [11]:
# Comparing example 2 (200ms)
filename = "C:/Users/sandra.oluoch/Desktop/slurm-down/golgi-200ms-ex2v8.1.tiff"
reader = AICSImage(filename)
IMG_ex2_200ms = reader.data.astype(np.float32)

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 0


In [16]:
ex2_similarity =2*np.sum(~np.logical_xor(IMG_ex2_50ms,IMG_ex2_200ms))/(IMG_ex2_50ms.size+IMG_ex2_200ms.size)
ex2_similarity

0.9998803125

<h3> Example 3 <h3> 

In [41]:
# Comparing example 3 (50ms)
filename = "C:/Users/sandra.oluoch/Desktop/slurm-down/golgi-50ms-ex3v9.2.tiff"
reader = AICSImage(filename)
IMG_ex3_50ms = reader.data.astype(np.float32)

filename_struct = "C:/Users/sandra.oluoch/Desktop/slurm-down/EX3_struct1_50ms.tiff"
reader_struct = AICSImage(filename_struct)
struct1_50ms = reader_struct.data.astype(np.float32)

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 0
Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 6


In [12]:
# Comparing example 3 (200ms)
filename = "C:/Users/sandra.oluoch/Desktop/slurm-down/golgi-200ms-ex3v9TEST.tiff"
reader = AICSImage(filename)
IMG_ex3_200ms = reader.data.astype(np.float32)

filename_struct = "C:/Users/sandra.oluoch/Desktop/slurm-down/EX3_struct1_200ms.tiff"
reader_struct = AICSImage(filename_struct)
struct1_200ms = reader_struct.data.astype(np.float32)

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 0
Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 6


In [40]:
# Matheus's version
ex3_similarity =2*np.sum(~np.logical_xor(IMG_ex3_50ms,IMG_ex3_200ms))/(IMG_ex3_50ms.size+IMG_ex3_200ms.size)
ex3_similarity

0.999679837962963

<h3> Example 5 <h3>

In [23]:
# Comparing example 2 (50ms)
filename = "C:/Users/sandra.oluoch/Desktop/slurm-down/golgi-50ms-ex5v8.tiff"
reader = AICSImage(filename)
IMG_ex5_50ms = reader.data.astype(np.float32)

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 0


In [24]:
# Comparing example 2 (200ms)
filename = "C:/Users/sandra.oluoch/Desktop/slurm-down/golgi-200ms-ex5v8.1.tiff"
reader = AICSImage(filename)
IMG_ex5_200ms = reader.data.astype(np.float32)

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 0


In [25]:
ex5_similarity =2*np.sum(~np.logical_xor(IMG_ex5_50ms,IMG_ex5_200ms))/(IMG_ex5_50ms.size+IMG_ex5_200ms.size)
ex5_similarity

0.9998991898148148

In [42]:
# Alternate version 
def jaccard_binary(x,y):
    """A function for finding the similarity between two binary vectors"""
    intersection = np.logical_and(x, y)
    union = np.logical_or(x, y)
    similarity = 2*(intersection.sum() / float(union.sum()))
    return similarity

In [43]:
jaccard_binary(struct1_50ms,struct1_200ms)

0.37914386868361766

<h3> F1-Score Calculation (Max Projection) <h3>

In [215]:
filename_50ms = "C:/Users/sandra.oluoch/Desktop/slurm-down/maxproj-seg-50ms-ex7v9.2.tiff"
reader = AICSImage(filename_50ms) 
IMG_50maxproj = reader.data.astype(np.float32)
IMG_50maxproj.shape

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 0


(1, 1, 1, 1200, 1800)

In [216]:
filename_200ms = "C:/Users/sandra.oluoch/Desktop/slurm-down/maxproj-seg-200ms-ex7v9.3.tiff"
reader = AICSImage(filename_200ms) 
IMG_200maxproj = reader.data.astype(np.float32)
IMG_200maxproj.shape

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 0


(1, 1, 1, 1200, 1800)

In [217]:
# reducing dimensions for f1-score

IMG50_squeeze1 = np.squeeze(IMG_50maxproj)
IMG200_squeeze1 = np.squeeze(IMG_200maxproj)

IMG200_squeeze1.shape

(1200, 1800)

In [218]:
from sklearn.preprocessing import MultiLabelBinarizer
binarizer = MultiLabelBinarizer()

# This should be your original approach
#binarizer.fit(your actual true output consisting of all labels)

# In this case, I am considering only the given labels.
binarizer.fit(IMG200_squeeze1)

f1_score(binarizer.transform(IMG200_squeeze1), 
         binarizer.transform(IMG50_squeeze1), 
         average='macro')

# code from: https://stackoverflow.com/questions/43198613/scikit-learn-f1-score-for-list-of-strings

0.8842538190364277

<h3> F1-Score Calculation (3D Image) <h3>

In [153]:
filename_50ms = "C:/Users/sandra.oluoch/Desktop/slurm-down/golgi-50ms-ex3v9.2.tiff"
reader = AICSImage(filename_50ms) 
IMG_50 = reader.data.astype(np.float32)
IMG_50.shape

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 0


(1, 1, 40, 1200, 1800)

In [155]:
filename_200ms = "C:/Users/sandra.oluoch/Desktop/slurm-down/golgi-50ms-ex3v9.2.tiff"
reader = AICSImage(filename_200ms) 
IMG_200 = reader.data.astype(np.float32)
IMG_200.shape

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 0


(1, 1, 40, 1200, 1800)

In [156]:
# reducing dimensions for f1-score

IMG50_squeeze1 = np.squeeze(IMG_50)
IMG200_squeeze1 = np.squeeze(IMG_200)

IMG200_squeeze1.shape

(40, 1200, 1800)

In [189]:
from sklearn.preprocessing import MultiLabelBinarizer
binarizer = MultiLabelBinarizer()

# This should be your original approach
#binarizer.fit(your actual true output consisting of all labels)

# In this case, I am considering only the given labels.
binarizer.fit(y1[20])

f1_score(binarizer.transform(y1[20]), 
         binarizer.transform(x1[20]), 
         average='macro')

1.0

<h3> Counting Segmented Pieces <h3>

In [263]:
from skimage import data, util
from skimage.measure import label, regionprops

img = util.img_as_ubyte(data.coins()) > 110
label_img = label(img, connectivity=img.ndim)

# define pixelcount function
def pixelcount(regionmask):
    return np.sum(regionmask)

props = regionprops(label_img, extra_properties=(pixelcount,))
props[0].pixelcount

7741

In [267]:
props2 = regionprops(IMG200_squeeze1.astype(int))
props2 = regionprops(IMG200_squeeze1.astype(int), extra_properties=(pixelcount,))
props2[0].pixelcount

3269